In [ ]:
import numpy as np
import pandas as pd
import torch
import os
import random
from shutil import copyfile
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torchvision
import torch.optim as optim
import torchvision.models as models
from tqdm import tqdm
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision import datasets
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, f1_score, auc, precision_score, recall_score
import matplotlib.pyplot as plt
import torchvision.models as models
from PIL import Image
import cv2
from torchvision.utils import save_image
from true_classify import *
from Utils import *
from anonymization_methods import *
from datasets import *
from torchvision.transforms.functional import to_pil_image
from collections import Counter
import matplotlib.image as mpimg
import time
import matplotlib.pyplot as plt
import xlrd
import openpyxl

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# define training and testing data directories
source_path = 'D:/Younas_Work/D2_Final/Original/All FR/train'

class_names = [folder for folder in os.listdir(source_path) if os.path.isdir(os.path.join(source_path, folder))]
num_classes = len(class_names)

file_list = os.listdir(source_path)
model_dir = 'Convnext_pretrained_younas.pt'

output_path = 'D:/Younas_Work/D2_Final/Anonymized/Expite/Simple_Pix_Old_15/Face_Recognition'

save_roc_dir = 'D:/Younas_Work/D2_Final/Results_Final/ROC/ROC Plots Mouth_Masked/'
excel_file_path = 'D:/Younas_Work/D2_Final/Results_Final/Excel_Sheets/Mouth_Masked.xlsx'

In [ ]:

# Define a new transform with additional data augmentations
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
model = models.convnext_base(pretrained=True)
model.classifier[2]=nn.Linear(1024,num_classes)

model.load_state_dict(torch.load('Convnext_pretrained_younas.pt'))
model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.001)

In [ ]:
# We will try a fixed q in each iteration
q=90

In [ ]:
#Setting Parameter values

# we will put all the values to highest possible to minimize the changes.

#For Simple Blur
blur_kernel_size = 3

#For DP Blur
#block
b = 16
#m & eps are for privacy budget. Larger m = more noise
m = 2
#eps is the privacy parameter in DP, smaller eps means higher noise and more distortion
eps = 100
#K is kernel size for gaussian blur. Height and width of the Gaussian kernel
k = 9
#SD of Gaussian blur. Larger sigma means more blurring
sigma = 0

#For Simple Pixelate
pixel_size = 3


#For DP Pixelate
#b, m and eps are the same for DP pixelate and DP Blur

In [ ]:
# start_time = time.time()
# new_batch_size = 1
# iteration_out_path = 'D:/Younas_Work/SOTA/DP/'
# new_test_path = iteration_out_path
# our_test_loader = create_test_loader(new_test_path, new_batch_size)
# final_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader, excel_file_path, save_roc_dir)
# prev_acc = final_acc
# print(prev_acc*100)
# end_time = time.time()
# overall_execution_time = end_time - start_time
# print(f"\nOverall execution time : {overall_execution_time} seconds")

In [ ]:
start_time = time.time()
new_batch_size = 1
iteration_out_path = 'D:/Younas_Work/D2_Final/Anonymized/Expite/Simple_Pix_Old_15/Face_Recognition/Iteration_15'
new_test_path = iteration_out_path
our_test_loader = create_test_loader(new_test_path, new_batch_size)
final_acc, correct_examples, labels, logits = test_images_classification(model, device, our_test_loader, excel_file_path, save_roc_dir)
prev_acc = final_acc
print(prev_acc*100)

end_time = time.time()
acc_time = end_time - start_time
print(f"\nAcc execution time: {acc_time} seconds")

start_time = time.time()

# If the number of iterations crashed your system, try the iterations in batches (batch of 5)
for itera in range(20):
    # Iterate through all correct examples
    for i in range(len(correct_examples)):
        x, correct_label, prediction = correct_examples[i], labels[i], logits[i]
        y = get_second_largest(logits[i])
        y = torch.tensor([y]).to(device)

        # Create the output directory for this iteration
        iteration_out_path = f"{output_path}/Iteration_{itera}/"

        class_label = labels[i]
        class_subfolder = class_names[class_label]            

        # Create subfolder based on the class name
    #         class_subfolder = class_names[correct_label.item()]

        class_output_path = os.path.join(iteration_out_path, class_subfolder)
        os.makedirs(class_output_path, exist_ok=True)

        # Clone the original image and enable gradient computation
        annonymized_image = x.clone()
        annonymized_image.requires_grad = True

        # Calculate the loss based on the model, image, and criterion
        output, loss = calculate_loss(model, annonymized_image, y, criterion)


        if(output.item() == labels[i].item()):
            model.zero_grad()
            loss.backward()
            img_grad = annonymized_image.grad.data

    # Call different perturbation functions here
#             perturbed = create_blurred_image(x, blur_kernel_size)
            perturbed = create_pixelated_image(x, pixel_size)
#             perturbed = create_dp_blurred_image(x, b, m, eps, k, sigma)
#            perturbed = create_dp_pixelated_image(x, b, m, eps)


            # Calculate noise based on the original and perturbed images
            noise = calculate_noise(x, perturbed, device)

            # Optimize the gradients using the quantile value (q)
            optimized_gradients = optimize_gradients(img_grad, q)
            optimized_gradients = optimized_gradients.to(device)

            # Update the noise using optimized gradients
            updated_noise = noise * optimized_gradients

            # Create the anonymized image by subtracting/adding the updated noise from the original image
            annonymized_image = annonymized_image - updated_noise

            # Save the anonymized image to the output directory
            save_image(annonymized_image, i, correct_label, class_output_path, pixel_size, q)  
        else:
            save_image(annonymized_image, i, correct_label, class_output_path, pixel_size, q)

end_time = time.time()
Anon_execution_time = end_time - start_time
print(f"\nAnon Time: {Anon_execution_time} seconds")
print(f"All images processed for iteration number {itera}.")